In [1]:
import os
import subprocess
import sys
import glob
sys.path.insert(0, '../scripts')
import transcribe_flac as transcribe

import numpy as np
import pandas as pd
import ffmpeg
import urllib.request
import re
import dask.dataframe as ddf
import time

In [2]:
def download_via_link(uri, directory):
    '''Downloads file at the link to a given directory.'''
    path, file_name = os.path.split(uri)
    file_name = re.sub("[&?=,']", "_", file_name) # make sure valid file name
    file_fp = f"{directory}/{file_name}"
    print(f"Downloading to {file_fp}")
    try:
        urllib.request.urlretrieve(uri, file_fp)
    except Exception as e:
        print(e)
        
    return file_name
    
    
# download_via_link("http://mycmag.kantarmediana.com/KMIcmagvidbin2/GOV_AK_FFAKF_SKYROCKETED.wmv", "../data/tv-ads-kantar")

def wmv_to_flac(vid_fp):
    '''Extracts audio from a WMV video file into a FLAC file of same name.'''
    path, file = os.path.split(vid_fp)
    audio_name = f"{file[:-4]}.flac"
    audio_fp = f"{path}/{audio_name}"
    
    ffmpeg_cmd = f"ffmpeg -y -i {vid_fp} -vn -f flac -ar 48000 -ab 128000 -ac 1 {audio_fp}"
    print(f"Converting to {audio_fp}")
    try:
        os.system(ffmpeg_cmd) # ret is what is returns on the command line
    except Exception as e:
        print(f"{vid_fp} ==> {e}")
    
    return audio_name

media_folder = "../data/tv-ads-kantar"

In [ ]:
tv_ads = pd.read_csv('../data/tv_ads_attack.csv').iloc[0:5]
media_folder = "../data/tv-ads-kantar"
tv_ads['vid_file'] = tv_ads.link.apply(lambda x: download_via_link(x, media_folder))
tv_ads['audio_file'] = tv_ads.vid_file.apply(lambda x: wmv_to_flac(f"{media_folder}/{x}"))
tv_ads['text_transcription'] = tv_ads.audio_file.apply(lambda x: transcribe.transcribe_file(f"{media_folder}/{x}"))
tv_ads

In [ ]:
# https://towardsdatascience.com/trying-out-dask-dataframes-in-python-for-fast-data-analysis-in-parallel-aa960c18a915
# def benchmark(function, function_name):
#     start = time.time()
#     function()
#     end = time.time()
#     print("{0} seconds for {1}".format((end - start), function_name))

# def apply_downloads_old():
#     df['vid_file'] = df.link.apply(lambda x: download_via_link(x, media_folder))
# def apply_downloads():
#     dd['vid_file'] = dd.link.apply(lambda x: download_via_link(x, media_folder)).compute()
    

# benchmark(apply_downloads_old, 'apply_downloads_old')
# benchmark(apply_downloads, 'apply_downloads')


In [ ]:
N = 8
tv_ads = pd.read_csv('../data/tv_ads_attack.csv')
dd_tv_ads = ddf.from_pandas(tv_ads, npartitions=N)

In [ ]:
start = time.time()
print(f"Start download of {tv_ads.shape[0]} videos at {start}")

media_folder = "../data/tv-ads-kantar"

dd_tv_ads['vid_file'] = dd_tv_ads.link.apply(lambda x: download_via_link(x, media_folder)).compute()

end = time.time()
print(f"Finished in  {end-start} seconds")

In [ ]:
start = time.time()
print(f"Start download of {tv_ads.shape[0]} videos at {start}")

dd_tv_ads['audio_file'] = dd_tv_ads.vid_file.apply(lambda x: wmv_to_flac(f"{media_folder}/{x}")).compute()

end = time.time()
print(f"Finished in  {end-start} seconds")

In [ ]:
start = time.time()
# print(f"Start download of {.shape[0]} videos at {start}")
media_folder = "../data/tv-ads-kantar"
dd['text_transcription'] = dd.audio_file.apply(lambda x: transcribe.transcribe_file(f"{media_folder}/{x}"), meta='str').compute()
# tv_ads['text_transcription'] = tv_ads.audio_file.apply(lambda x: transcribe.transcribe_file(f"{media_folder}/{x}"))
end = time.time()
print(f"Finished in  {end-start} seconds")

Transcribing ../data/tv-ads-kantar/GENAMBLY_NJ_BATEMAN_SIMON_CALIGUIRE_PRINCETON_ELITIST.flac
Transcribing ../data/tv-ads-kantar/GOV_MD_HOGAN_EQUAL_PAY_LAW.flacTranscribing ../data/tv-ads-kantar/GOV_TX_ABBOTT_BEST_BUSINESS_CLIMATE.flacTranscribing ../data/tv-ads-kantar/HOUSE_IA01_CHANGENOW_WORST_OF_WASHINGTON_PREEXISTING_CONDITIONS.flac


Transcribing ../data/tv-ads-kantar/GOV_MD_HOGAN_JIMMY.flac
Transcribing ../data/tv-ads-kantar/GOV_AK_AOD_FLIP_FLOPPED.flac
Transcribing ../data/tv-ads-kantar/HOUSE_IA01_CLF_DREAM.flac
Transcribing ../data/tv-ads-kantar/GOV_TX_ABBOTT_BEST_BUSINESS_CLIMATE_REV.flac
Transcribing ../data/tv-ads-kantar/GOV_MD_HOGAN_KIMBERLY_ELLIS.flac
Transcribing ../data/tv-ads-kantar/GOV_AK_AOD_HURT_YOU.flac
Transcribing ../data/tv-ads-kantar/GOV_TX_ABBOTT_BEST_BUSINESS_CLIMATE_REV_2.flac
Transcribing ../data/tv-ads-kantar/HOUSE_IA01_CLF_LIBERAL.flac
Transcribing ../data/tv-ads-kantar/GOV_MD_HOGAN_MARYLAND_STRONG.flac
Transcribing ../data/tv-ads-kantar/GOV_AK_BEGICH_HIS_

In [ ]:
dd_tv_ads.to_csv("../data/tv_ads_attack_dd*.csv", index=False)

In [ ]:
dd

In [3]:
dd = ddf.read_csv("../data/tv_ads_attack_dd*.csv")
dd

,alt,link,ad_tone,categorystate,level,race,vid_file
npartitions=8,,,,,,,
,object,object,object,object,object,object,object
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [4]:
dd['audio_file'] = dd.vid_file.apply(lambda x: f"{x[:-4]}.flac")
dd

C:\Users\fredcorpuz06\Anaconda3\envs\audioVisual\lib\site-packages\dask\dataframe\core.py:2345: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('vid_file', 'object'))

  warnings.warn(meta_warning(meta))


,alt,link,ad_tone,categorystate,level,race,vid_file,audio_file
npartitions=8,,,,,,,,
,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


In [ ]:
dd.to_csv("../data/fred*.csv", index=False)